In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## NASHVILLE CRIME DF

In [4]:
nashvillecrime_df=pd.read_csv("..\data\Metro_Nashville_Police_Department_Incidents_20240215.csv")

In [5]:
nashvillecrime_df.head()

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Victim Description,Victim Gender,Victim Race,Victim Ethnicity,Victim County Resident,Mapped Location,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS)
0,20240103838_23,20240103838,D,DISPATCHED,O,OPEN,Open,10/18/2023 08:00:00 PM,02/14/2024 07:27:00 PM,LAFAYETTE ST,...,SOCIETY,NaN,NaN,NaN,NaN,POINT (-86.76 36.15),1.0,1.0,20.0,11.0
1,20240103838_12,20240103838,D,DISPATCHED,O,OPEN,Open,10/18/2023 08:00:00 PM,02/14/2024 07:27:00 PM,LAFAYETTE ST,...,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,POINT (-86.76 36.15),1.0,1.0,20.0,11.0
2,20240103838_11,20240103838,D,DISPATCHED,O,OPEN,Open,10/18/2023 08:00:00 PM,02/14/2024 07:27:00 PM,LAFAYETTE ST,...,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,NON RESIDENT,POINT (-86.76 36.15),1.0,1.0,20.0,11.0
3,20240103523_11,20240103523,T,NaN,O,OPEN,Open,06/01/2023 12:00:00 PM,02/14/2024 01:36:00 PM,GLENPARK DR,...,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.71 36.11),1.0,1.0,6.0,22.0
4,20240102993_11,20240102993,D,DISPATCHED,O,OPEN,Open,04/06/2023 06:23:00 PM,02/14/2024 09:18:00 AM,CHARLES E DAVIS BLVD,...,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,RESIDENT,POINT (-86.76 36.15),1.0,1.0,20.0,11.0


In [6]:
nashvillecrime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122764 entries, 0 to 122763
Data columns (total 35 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Primary Key                      122764 non-null  object 
 1   Incident Number                  122764 non-null  int64  
 2   Report Type                      122734 non-null  object 
 3   Report Type Description          112165 non-null  object 
 4   Incident Status Code             122764 non-null  object 
 5   Incident Status Description      122753 non-null  object 
 6   Investigation Status             122764 non-null  object 
 7   Incident Occurred                122764 non-null  object 
 8   Incident Reported                122764 non-null  object 
 9   Incident Location                120353 non-null  object 
 10  Latitude                         118035 non-null  float64
 11  Longitude                        118035 non-null  float64
 12  ZI

In [7]:
nashvillecrime_df['Incident Occurred']

0         10/18/2023 08:00:00 PM
1         10/18/2023 08:00:00 PM
2         10/18/2023 08:00:00 PM
3         06/01/2023 12:00:00 PM
4         04/06/2023 06:23:00 PM
                   ...          
122759    08/13/2023 11:50:00 PM
122760    05/15/2023 08:00:00 AM
122761    03/01/2023 08:00:00 PM
122762    03/17/2023 08:21:00 PM
122763    02/27/2023 12:15:00 PM
Name: Incident Occurred, Length: 122764, dtype: object

In [8]:
nashvillecrime_df['Date'] = pd.to_datetime(nashvillecrime_df['Incident Occurred']).dt.date
nashvillecrime_df['Time'] = pd.to_datetime(nashvillecrime_df['Incident Occurred']).dt.time

print(nashvillecrime_df)

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\1469182027.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nashvillecrime_df['Date'] = pd.to_datetime(nashvillecrime_df['Incident Occurred']).dt.date
C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\1469182027.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nashvillecrime_df['Time'] = pd.to_datetime(nashvillecrime_df['Incident Occurred']).dt.time


           Primary Key  Incident Number Report Type Report Type Description  \
0       20240103838_23      20240103838           D              DISPATCHED   
1       20240103838_12      20240103838           D              DISPATCHED   
2       20240103838_11      20240103838           D              DISPATCHED   
3       20240103523_11      20240103523           T                     NaN   
4       20240102993_11      20240102993           D              DISPATCHED   
...                ...              ...         ...                     ...   
122759  20200478485_11      20200478485           D              DISPATCHED   
122760  20200306667_11      20200306667           D              DISPATCHED   
122761  20200130530_11      20200130530           T                     NaN   
122762  20200020059_11      20200020059           O                     NaN   
122763  20130140658_11      20130140658           T                     NaN   

       Incident Status Code Incident Status Descrip

## WEATHER DF

In [9]:
weather2023_df=pd.read_excel(r"C:\Users\antoi\nss-data-analytics\Projects\Nashville_Crime_and_Weather\data\Nashville_Weather_2023.xlsx")

In [10]:
weather2023_df

,Day,High,Low,Precip.
0,2023-01-01,70,45,0.00
1,2023-01-02,75,53,0.04
2,2023-01-03,71,58,1.53
3,2023-01-04,65,45,0.07
4,2023-01-05,56,36,0.00
...,...,...,...,...
359,2023-12-27,53,32,0.00
360,2023-12-28,44,31,0.00
361,2023-12-29,41,37,0.00
362,2023-12-30,41,31,0.00


In [11]:
weather2023_df.rename(columns={"Day":"Date"},inplace=True)

In [12]:
weather2023_df

,Date,High,Low,Precip.
0,2023-01-01,70,45,0.00
1,2023-01-02,75,53,0.04
2,2023-01-03,71,58,1.53
3,2023-01-04,65,45,0.07
4,2023-01-05,56,36,0.00
...,...,...,...,...
359,2023-12-27,53,32,0.00
360,2023-12-28,44,31,0.00
361,2023-12-29,41,37,0.00
362,2023-12-30,41,31,0.00


## MERGE

In [13]:
#converting weather df [date] to object
weather2023_df['Date'] = pd.to_datetime(weather2023_df['Date']).dt.date

In [14]:
nashvillecrime_weather=pd.merge( nashvillecrime_df, weather2023_df, on="Date", how="inner")
print(nashvillecrime_weather)

           Primary Key  Incident Number Report Type Report Type Description  \
0       20240103838_23      20240103838           D              DISPATCHED   
1       20240103838_12      20240103838           D              DISPATCHED   
2       20240103838_11      20240103838           D              DISPATCHED   
3       20230760496_11      20230760496           D              DISPATCHED   
4       20230756034_11      20230756034           T                     NaN   
...                ...              ...         ...                     ...   
122399  20230023455_11      20230023455           D              DISPATCHED   
122400  20230023444_11      20230023444           D              DISPATCHED   
122401  20230023407_11      20230023407           D              DISPATCHED   
122402  20230022967_11      20230022967           D              DISPATCHED   
122403  20220402711_11      20220402711           S                 SUSPECT   

       Incident Status Code Incident Status Descrip

In [15]:
nashvillecrime_weather.columns= nashvillecrime_weather.columns.str.strip()

In [16]:
nashvillecrime_weather['Offense Description'].value_counts()

Offense Description
POLICE INQUIRY                                         22574
BURGLARY- MOTOR VEHICLE                                 9448
SIMPLE ASSLT                                            6332
VEHICLE THEFT                                           5284
FOUND PROPERTY                                          4870
                                                       ...  
DRIVING UNDER THE INFLUENCE, ALCOHOL                       1
MILITARY PROP.- LOSS/DAMAGE/DEST. $1,000 OR LESS           1
DRAWING PROPERTY TRANSFER DOCUMENTS W/OUT OWNERSHIP        1
Poss.w/Int-Cont.Sub.-Amphetamine 26-299 gr                 1
IMPAIRED ADULT AGED 60 OR > - ABUSE                        1
Name: count, Length: 398, dtype: int64

In [17]:
#getting average temp 

nashvillecrime_weather['Average Temp'] = (nashvillecrime_weather.High + nashvillecrime_weather.Low ) / 2
print(nashvillecrime_weather)

           Primary Key  Incident Number Report Type Report Type Description  \
0       20240103838_23      20240103838           D              DISPATCHED   
1       20240103838_12      20240103838           D              DISPATCHED   
2       20240103838_11      20240103838           D              DISPATCHED   
3       20230760496_11      20230760496           D              DISPATCHED   
4       20230756034_11      20230756034           T                     NaN   
...                ...              ...         ...                     ...   
122399  20230023455_11      20230023455           D              DISPATCHED   
122400  20230023444_11      20230023444           D              DISPATCHED   
122401  20230023407_11      20230023407           D              DISPATCHED   
122402  20230022967_11      20230022967           D              DISPATCHED   
122403  20220402711_11      20220402711           S                 SUSPECT   

       Incident Status Code Incident Status Descrip

## CLEANING

In [18]:
# dropping police inquiry 
nashvillecrime_weather[nashvillecrime_weather['Offense Description'] != 'POLICE INQUIRY']

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
0,20240103838_23,20240103838,D,DISPATCHED,O,OPEN,Open,10/18/2023 08:00:00 PM,02/14/2024 07:27:00 PM,LAFAYETTE ST,...,1.0,1.0,20.0,11.0,2023-10-18,20:00:00,73,41,0.00,57.0
1,20240103838_12,20240103838,D,DISPATCHED,O,OPEN,Open,10/18/2023 08:00:00 PM,02/14/2024 07:27:00 PM,LAFAYETTE ST,...,1.0,1.0,20.0,11.0,2023-10-18,20:00:00,73,41,0.00,57.0
2,20240103838_11,20240103838,D,DISPATCHED,O,OPEN,Open,10/18/2023 08:00:00 PM,02/14/2024 07:27:00 PM,LAFAYETTE ST,...,1.0,1.0,20.0,11.0,2023-10-18,20:00:00,73,41,0.00,57.0
3,20230760496_11,20230760496,D,DISPATCHED,O,OPEN,Open,10/18/2023 04:00:00 PM,12/20/2023 12:00:00 PM,ELM HILL,...,1.0,1.0,8.0,11.0,2023-10-18,16:00:00,73,41,0.00,57.0
6,20230735108_11,20230735108,S,SUSPECT,O,OPEN,Open,10/18/2023 04:15:00 PM,12/08/2023 04:15:00 PM,BELMONT BLVD,...,1.0,1.0,18.0,36.0,2023-10-18,16:15:00,73,41,0.00,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122397,20230023469_11,20230023469,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,01/12/2023 01:00:00 AM,01/12/2023 02:36:00 AM,1311 1311,...,1.0,1.0,13.0,16.0,2023-01-12,01:00:00,69,40,0.31,54.5
122398,20230023456_11,20230023456,D,DISPATCHED,O,OPEN,Open,01/12/2023 12:30:00 AM,01/12/2023 02:55:00 AM,HAMPTON ST,...,1.0,1.0,18.0,11.0,2023-01-12,00:30:00,69,40,0.31,54.5
122400,20230023444_11,20230023444,D,DISPATCHED,O,OPEN,Open,01/12/2023 12:30:00 AM,01/12/2023 01:26:00 AM,ANTHES DR,...,1.0,1.0,14.0,19.0,2023-01-12,00:30:00,69,40,0.31,54.5
122402,20230022967_11,20230022967,D,DISPATCHED,U,UNFOUNDED,Closed,01/12/2023 07:07:00 AM,01/12/2023 07:07:00 AM,264 264,...,1.0,2.0,17.0,12.0,2023-01-12,07:07:00,69,40,0.31,54.5


In [19]:
#dropping unfounded from 'Incident Status'
nashvillecrime_weather=nashvillecrime_weather[nashvillecrime_weather['Incident Status Description'] != 'UNFOUNDED']

## ANALYSIS- Offenses Above and Below 70 degrees average

In [20]:
#What crimes are committed when the Average temp is 70 and above
crime_above_70 = nashvillecrime_weather.loc[nashvillecrime_weather["Average Temp"] >= 70, "Offense Description"]

crime_above_70.value_counts()

Offense Description
BURGLARY- MOTOR VEHICLE                                         4291
SIMPLE ASSLT                                                    2694
VEHICLE THEFT                                                   2414
ASSAULT- FEAR OF BODILY INJURY                                  1869
DAMAGE PROP - PRIVATE                                           1789
                                                                ... 
CHILD NEGLECT - EXPOSE TO CONTR. SUBST. & CHILD 9-17               1
WEAPON - CARRYING AFTER FEDERAL RESTRICTIONS                       1
Assault, Officer/Responder - Agg - Serious Injury - Reckless       1
BURGLARY, ACCESSORY TO                                             1
IMITATION CONTROLLED SUBSTANCE - USE                               1
Name: count, Length: 326, dtype: int64

In [21]:
crime_vehicle=nashvillecrime_weather.loc[nashvillecrime_weather['Offense Description'].isin(['BURGLARY- MOTOR VEHICLE'])]
crime_vehicle

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
9,20230699855_11,20230699855,W,WITNESS,O,OPEN,Open,10/18/2023 06:45:00 AM,11/22/2023 10:08:00 AM,MCGAVOCK,...,1.0,1.0,8.0,38.0,2023-10-18,06:45:00,73,41,0.00,57.0
21,20230639622_11,20230639622,W,WITNESS,O,OPEN,Open,10/18/2023 07:00:00 AM,10/26/2023 12:38:00 PM,REISCHA DR,...,1.0,1.0,6.0,3.0,2023-10-18,07:00:00,73,41,0.00,57.0
30,20230634676_11,20230634676,T,NaN,O,OPEN,Open,10/18/2023 11:00:00 PM,10/24/2023 03:39:00 PM,PATRICIA DR,...,1.0,1.0,6.0,22.0,2023-10-18,23:00:00,73,41,0.00,57.0
31,20230634621_11,20230634621,D,DISPATCHED,O,OPEN,Open,10/18/2023 09:00:00 PM,10/24/2023 03:22:00 PM,2ND AV S,...,1.0,1.0,20.0,29.0,2023-10-18,21:00:00,73,41,0.00,57.0
32,20230634297_11,20230634297,T,NaN,O,OPEN,Open,10/18/2023 01:30:00 PM,10/24/2023 12:36:00 PM,JAMES ROBERTSON PKWY,...,1.0,1.0,20.0,29.0,2023-10-18,13:30:00,73,41,0.00,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122227,20230025455_11,20230025455,D,DISPATCHED,A,CLEARED BY ARREST,Closed,01/12/2023 08:50:00 PM,01/13/2023 12:49:00 AM,808 808,...,1.0,1.0,30.0,15.0,2023-01-12,20:50:00,69,40,0.31,54.5
122299,20230024784_11,20230024784,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:00:00 AM,01/12/2023 03:51:00 PM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:00:00,69,40,0.31,54.5
122340,20230024459_11,20230024459,T,NaN,O,OPEN,Open,01/12/2023 01:30:00 AM,01/12/2023 12:10:00 PM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:30:00,69,40,0.31,54.5
122355,20230024250_11,20230024250,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:40:00 AM,01/12/2023 10:22:00 AM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:40:00,69,40,0.31,54.5


In [22]:
#What crimes are committed when the average temp is below 70 degrees?
crime_below_70 = nashvillecrime_weather.loc[nashvillecrime_weather["Average Temp"] <=69, "Offense Description"]
crime_below_70.value_counts()

Offense Description
BURGLARY- MOTOR VEHICLE                                       4937
SIMPLE ASSLT                                                  3466
VEHICLE THEFT                                                 2598
ASSAULT- FEAR OF BODILY INJURY                                2278
DAMAGE PROP - PRIVATE                                         2085
                                                              ... 
DRIVING UNDER THE INFLUENCE - 3RD OFFENSE                        1
MARIJUANA CONCENTRATE-MANUFACTURE                                1
COUNTERFEIT LOGO - USE OR SELL-  $2,500 OR > BUT < $10,000       1
DESTRUCTION OF PAPERS TO DEFAUD- >$1,000 BUT <$2,500             1
ASSAULT OFFICER-AGG - STRANGULATION - INT/KN                     1
Name: count, Length: 341, dtype: int64

##### From my analysis of the average temp, it seems more crimes are committed when it's below 70 degrees vs when it's hotter. 

## Analysis- Burglary- Motor Vehicle 

In [23]:
crime_vehicle =nashvillecrime_weather.loc[nashvillecrime_weather['Offense Description'].isin(['BURGLARY- MOTOR VEHICLE'])]
crime_vehicle

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
9,20230699855_11,20230699855,W,WITNESS,O,OPEN,Open,10/18/2023 06:45:00 AM,11/22/2023 10:08:00 AM,MCGAVOCK,...,1.0,1.0,8.0,38.0,2023-10-18,06:45:00,73,41,0.00,57.0
21,20230639622_11,20230639622,W,WITNESS,O,OPEN,Open,10/18/2023 07:00:00 AM,10/26/2023 12:38:00 PM,REISCHA DR,...,1.0,1.0,6.0,3.0,2023-10-18,07:00:00,73,41,0.00,57.0
30,20230634676_11,20230634676,T,NaN,O,OPEN,Open,10/18/2023 11:00:00 PM,10/24/2023 03:39:00 PM,PATRICIA DR,...,1.0,1.0,6.0,22.0,2023-10-18,23:00:00,73,41,0.00,57.0
31,20230634621_11,20230634621,D,DISPATCHED,O,OPEN,Open,10/18/2023 09:00:00 PM,10/24/2023 03:22:00 PM,2ND AV S,...,1.0,1.0,20.0,29.0,2023-10-18,21:00:00,73,41,0.00,57.0
32,20230634297_11,20230634297,T,NaN,O,OPEN,Open,10/18/2023 01:30:00 PM,10/24/2023 12:36:00 PM,JAMES ROBERTSON PKWY,...,1.0,1.0,20.0,29.0,2023-10-18,13:30:00,73,41,0.00,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122227,20230025455_11,20230025455,D,DISPATCHED,A,CLEARED BY ARREST,Closed,01/12/2023 08:50:00 PM,01/13/2023 12:49:00 AM,808 808,...,1.0,1.0,30.0,15.0,2023-01-12,20:50:00,69,40,0.31,54.5
122299,20230024784_11,20230024784,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:00:00 AM,01/12/2023 03:51:00 PM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:00:00,69,40,0.31,54.5
122340,20230024459_11,20230024459,T,NaN,O,OPEN,Open,01/12/2023 01:30:00 AM,01/12/2023 12:10:00 PM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:30:00,69,40,0.31,54.5
122355,20230024250_11,20230024250,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:40:00 AM,01/12/2023 10:22:00 AM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:40:00,69,40,0.31,54.5


In [24]:
crime_vehicle['Average Temp'].value_counts()

Average Temp
48.5    263
82.5    257
79.5    252
72.5    235
79.0    225
       ... 
85.0     17
31.0     14
58.0     12
43.5     12
38.5     10
Name: count, Length: 101, dtype: int64

In [25]:
crime_vehicle['Date'].value_counts()

Date
2023-11-11    63
2023-10-07    58
2023-09-02    57
2023-09-04    57
2023-06-24    56
              ..
2023-02-28     8
2023-01-31     8
2023-02-12     8
2023-04-18     7
2023-01-30     5
Name: count, Length: 348, dtype: int64

In [26]:
crime_vehicle['Location Description'].value_counts()

Location Description
PARKING LOT, GARAGE                   6465
RESIDENCE, HOME                       1404
HIGHWAY, ROAD, ALLEY                   973
PARK                                   155
APARTMENT                              110
HOTEL, MOTEL, ETC.                      81
SERVICE, GAS STATION                    34
Park/Playground                         31
CONSTRUCTION SITE                       28
Auto Dealership New/Used                19
School-Elementary/Secondary             16
CONVENIENCE STORE                       12
CEMETERY                                10
BAR, NIGHT CLUB                         10
DEPARTMENT, DISCOUNT STORE               9
RESTAURANT                               9
SPECIALTY STORE                          8
COMMERCIAL, OFFICE BUILDING              7
Auto Salvage/Junkyard                    6
GROCERY, SUPERMARKET                     5
Arena/Stadium/Fairgrounds/Coliseum       4
OUTBUILDING                              3
HOSPITAL                         

In [27]:
crime_vehicle.pl=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['PARKING LOT, GARAGE'])]
crime_vehicle.pl

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\126927233.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_vehicle.pl=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['PARKING LOT, GARAGE'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
9,20230699855_11,20230699855,W,WITNESS,O,OPEN,Open,10/18/2023 06:45:00 AM,11/22/2023 10:08:00 AM,MCGAVOCK,...,1.0,1.0,8.0,38.0,2023-10-18,06:45:00,73,41,0.00,57.0
30,20230634676_11,20230634676,T,NaN,O,OPEN,Open,10/18/2023 11:00:00 PM,10/24/2023 03:39:00 PM,PATRICIA DR,...,1.0,1.0,6.0,22.0,2023-10-18,23:00:00,73,41,0.00,57.0
31,20230634621_11,20230634621,D,DISPATCHED,O,OPEN,Open,10/18/2023 09:00:00 PM,10/24/2023 03:22:00 PM,2ND AV S,...,1.0,1.0,20.0,29.0,2023-10-18,21:00:00,73,41,0.00,57.0
32,20230634297_11,20230634297,T,NaN,O,OPEN,Open,10/18/2023 01:30:00 PM,10/24/2023 12:36:00 PM,JAMES ROBERTSON PKWY,...,1.0,1.0,20.0,29.0,2023-10-18,13:30:00,73,41,0.00,57.0
34,20230629674_11,20230629674,D,DISPATCHED,O,OPEN,Open,10/18/2023 08:00:00 PM,10/22/2023 06:22:00 PM,JAMES ROBERTSON PKWY,...,1.0,1.0,20.0,29.0,2023-10-18,20:00:00,73,41,0.00,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122226,20230025459_11,20230025459,D,DISPATCHED,O,OPEN,Open,01/12/2023 06:00:00 PM,01/12/2023 11:00:00 PM,GALLATIN AV,...,1.0,1.0,13.0,4.0,2023-01-12,18:00:00,69,40,0.31,54.5
122227,20230025455_11,20230025455,D,DISPATCHED,A,CLEARED BY ARREST,Closed,01/12/2023 08:50:00 PM,01/13/2023 12:49:00 AM,808 808,...,1.0,1.0,30.0,15.0,2023-01-12,20:50:00,69,40,0.31,54.5
122299,20230024784_11,20230024784,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:00:00 AM,01/12/2023 03:51:00 PM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:00:00,69,40,0.31,54.5
122340,20230024459_11,20230024459,T,NaN,O,OPEN,Open,01/12/2023 01:30:00 AM,01/12/2023 12:10:00 PM,BRILEY PKWY,...,1.0,1.0,6.0,38.0,2023-01-12,01:30:00,69,40,0.31,54.5


In [28]:
crime_vehicle.pl['Average Temp'].mean()

65.01198762567672

In [29]:
crime_vehicle.pl['Precip.'].mean()

0.11498685228151585

In [30]:
crime_vehicle.pl['Date'].value_counts()

Date
2023-02-24    48
2023-04-08    48
2023-11-11    47
2023-09-02    46
2023-10-09    45
              ..
2023-02-12     4
2023-03-28     4
2023-04-06     3
2023-01-30     3
2023-03-02     3
Name: count, Length: 348, dtype: int64

In [31]:
crime_vehicle.res=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['RESIDENCE, HOME'])]
crime_vehicle.res

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\3412542488.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_vehicle.res=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['RESIDENCE, HOME'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
21,20230639622_11,20230639622,W,WITNESS,O,OPEN,Open,10/18/2023 07:00:00 AM,10/26/2023 12:38:00 PM,REISCHA DR,...,1.0,1.0,6.0,3.0,2023-10-18,07:00:00,73,41,0.00,57.0
46,20230625683_11,20230625683,T,NaN,O,OPEN,Open,10/18/2023 12:01:00 AM,10/20/2023 12:59:00 PM,PECAN ST,...,1.0,1.0,2.0,51.0,2023-10-18,00:01:00,73,41,0.00,57.0
191,20230620876_11,20230620876,T,NaN,O,OPEN,Open,10/18/2023 01:10:00 PM,10/18/2023 05:02:00 PM,ESTES RD,...,1.0,1.0,34.0,47.0,2023-10-18,13:10:00,73,41,0.00,57.0
481,20230324041_11,20230324041,D,DISPATCHED,O,OPEN,Open,06/01/2023 07:00:00 PM,06/02/2023 10:52:00 PM,RIVERSTONE CT,...,1.0,1.0,7.0,38.0,2023-06-01,19:00:00,88,67,0.00,77.5
487,20230323231_11,20230323231,T,NaN,O,OPEN,Open,06/01/2023 06:00:00 PM,06/02/2023 01:52:00 PM,WOODBURY FALLS CT,...,1.0,1.0,32.0,32.0,2023-06-01,18:00:00,88,67,0.00,77.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121965,20230106328_11,20230106328,D,DISPATCHED,O,OPEN,Open,02/19/2023 05:00:00 AM,02/19/2023 03:55:00 PM,TENNESSEE AVE,...,1.0,1.0,24.0,46.0,2023-02-19,05:00:00,63,44,0.00,53.5
121980,20230106238_11,20230106238,D,DISPATCHED,O,OPEN,Open,02/19/2023 12:00:00 AM,02/19/2023 02:28:00 PM,WOODBERRY DR,...,1.0,1.0,8.0,38.0,2023-02-19,00:00:00,63,44,0.00,53.5
121998,20230106181_11,20230106181,D,DISPATCHED,O,OPEN,Open,02/19/2023 02:00:00 AM,02/19/2023 01:49:00 PM,CROSSFIELD DR,...,1.0,1.0,8.0,38.0,2023-02-19,02:00:00,63,44,0.00,53.5
122141,20230032516_11,20230032516,D,DISPATCHED,O,OPEN,Open,01/12/2023 10:30:00 PM,01/16/2023 03:30:00 PM,JOYNER AVE,...,1.0,1.0,9.0,11.0,2023-01-12,22:30:00,69,40,0.31,54.5


In [32]:
crime_vehicle.res['Average Temp'].mean()

66.0925925925926

In [33]:
crime_vehicle.res['Precip.'].mean()

0.09923789173789174

In [34]:
crime_vehicle.res['Date'].value_counts()

Date
2023-06-04    18
2023-06-24    15
2023-05-25    14
2023-01-15    14
2023-02-07    13
              ..
2023-08-05     1
2023-05-19     1
2023-02-02     1
2023-06-25     1
2023-11-08     1
Name: count, Length: 326, dtype: int64

In [35]:
crime_vehicle.hra=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['HIGHWAY, ROAD, ALLEY'])]
crime_vehicle.hra

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\2659799564.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_vehicle.hra=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['HIGHWAY, ROAD, ALLEY'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
40,20230627590_11,20230627590,D,DISPATCHED,O,OPEN,Open,10/18/2023 06:30:00 PM,10/21/2023 10:31:00 AM,S 11TH ST,...,1.0,1.0,14.0,4.0,2023-10-18,18:30:00,73,41,0.00,57.0
363,20230581238_11,20230581238,D,DISPATCHED,O,OPEN,Open,06/01/2023 12:53:00 PM,09/29/2023 12:53:00 PM,HART ST,...,1.0,1.0,18.0,11.0,2023-06-01,12:53:00,88,67,0.00,77.5
594,20230321429_11,20230321429,D,DISPATCHED,O,OPEN,Open,06/01/2023 03:55:00 AM,06/01/2023 07:13:00 PM,FAIRBROOK DR,...,1.0,1.0,8.0,38.0,2023-06-01,03:55:00,88,67,0.00,77.5
724,20230320234_11,20230320234,D,DISPATCHED,O,OPEN,Open,06/01/2023 02:18:00 AM,06/01/2023 07:27:00 AM,WELLMAN DR,...,1.0,1.0,8.0,38.0,2023-06-01,02:18:00,88,67,0.00,77.5
1062,20230771564_11,20230771564,D,DISPATCHED,O,OPEN,Open,11/01/2023 09:00:00 AM,12/26/2023 10:11:00 PM,WATERFORD WAY,...,1.0,1.0,33.0,15.0,2023-11-01,09:00:00,51,30,0.00,40.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121162,20230075892_11,20230075892,D,DISPATCHED,O,OPEN,Open,02/03/2023 02:00:00 PM,02/05/2023 05:49:00 PM,BONNALYNN DR,...,1.0,2.0,7.0,44.0,2023-02-03,14:00:00,36,24,0.00,30.0
121194,20230073786_21,20230073786,D,DISPATCHED,O,OPEN,Open,02/03/2023 07:00:00 PM,02/04/2023 11:48:00 AM,PIANO WAY,...,1.0,1.0,2.0,16.0,2023-02-03,19:00:00,36,24,0.00,30.0
121210,20230073269_11,20230073269,D,DISPATCHED,O,OPEN,Open,02/03/2023 09:30:00 PM,02/04/2023 06:08:00 AM,FAIRFIELD AV,...,1.0,1.0,20.0,11.0,2023-02-03,21:30:00,36,24,0.00,30.0
121512,20230206995_11,20230206995,D,DISPATCHED,O,OPEN,Open,04/07/2023 10:30:00 PM,04/08/2023 02:03:00 PM,NaN,...,1.0,1.0,18.0,36.0,2023-04-07,22:30:00,60,47,0.02,53.5


In [36]:
crime_vehicle.hra['Average Temp'].mean()

65.97584789311408

In [37]:
crime_vehicle.hra['Precip.'].mean()

0.10040082219938334

In [38]:
crime_vehicle.hra['Date'].value_counts()

Date
2023-10-07    14
2023-11-11    14
2023-09-18    13
2023-10-03    13
2023-09-17    12
              ..
2023-08-24     1
2023-07-18     1
2023-10-12     1
2023-06-10     1
2023-02-19     1
Name: count, Length: 290, dtype: int64

In [39]:
crime_vehicle.pk=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['PARK'])]
crime_vehicle.pk

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\1626901515.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_vehicle.pk=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['PARK'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
3835,20230724923_11,20230724923,D,DISPATCHED,O,OPEN,Open,12/04/2023 01:45:00 PM,12/04/2023 04:28:00 PM,MURPHY RD,...,1.0,1.0,10.0,46.0,2023-12-04,13:45:00,58,40,0.01,49.0
3882,20230724588_11,20230724588,D,DISPATCHED,O,OPEN,Open,12/04/2023 01:58:00 PM,12/04/2023 01:58:00 PM,OLD HICKORY BLVD W,...,1.0,2.0,16.0,16.0,2023-12-04,13:58:00,58,40,0.01,49.0
4080,20230656432_11,20230656432,D,DISPATCHED,O,OPEN,Open,11/02/2023 04:10:00 PM,11/02/2023 11:50:00 PM,PARTHENON AV,...,1.0,1.0,25.0,46.0,2023-11-02,16:10:00,61,28,0.00,44.5
4506,20230642415_11,20230642415,D,DISPATCHED,O,OPEN,Open,10/27/2023 12:00:00 PM,10/27/2023 05:42:00 PM,LITTLE MARROWBONE RD,...,1.0,2.0,1.0,34.0,2023-10-27,12:00:00,81,66,0.00,73.5
4836,20230644595_11,20230644595,D,DISPATCHED,O,OPEN,Open,10/28/2023 02:00:00 PM,10/28/2023 06:49:00 PM,OLD HICKORY BLVD,...,1.0,2.0,27.0,26.0,2023-10-28,14:00:00,87,66,0.00,76.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119113,20230087909_11,20230087909,D,DISPATCHED,O,OPEN,Open,02/10/2023 11:45:00 AM,02/10/2023 02:08:00 PM,VAUGHN RD,...,1.0,2.0,34.0,32.0,2023-02-10,11:45:00,58,39,0.00,48.5
120303,20230115374_12,20230115374,D,DISPATCHED,O,OPEN,Open,02/23/2023 02:50:00 PM,02/23/2023 04:47:00 PM,HIGHWAY 100,...,1.0,2.0,34.0,32.0,2023-02-23,14:50:00,85,55,0.00,70.0
120304,20230115374_11,20230115374,D,DISPATCHED,O,OPEN,Open,02/23/2023 02:50:00 PM,02/23/2023 04:47:00 PM,HIGHWAY 100,...,1.0,2.0,34.0,32.0,2023-02-23,14:50:00,85,55,0.00,70.0
121382,20230071736_11,20230071736,D,DISPATCHED,O,OPEN,Open,02/03/2023 10:05:00 AM,02/03/2023 11:30:00 AM,VAUGHN RD,...,1.0,2.0,34.0,32.0,2023-02-03,10:05:00,36,24,0.00,30.0


In [40]:
crime_vehicle.pk['Average Temp'].mean()

63.193548387096776

In [41]:
crime_vehicle.pk['Precip.'].mean()

0.0716774193548387

In [42]:
crime_vehicle.pk['Date'].value_counts()

Date
2023-11-30    4
2023-03-06    4
2023-06-19    4
2023-03-07    3
2023-09-30    3
             ..
2023-11-28    1
2023-08-23    1
2023-06-10    1
2023-11-19    1
2023-04-07    1
Name: count, Length: 104, dtype: int64

In [43]:
crime_vehicle.apt=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['APARTMENT'])]
crime_vehicle.apt

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\593460987.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_vehicle.apt=crime_vehicle.loc[crime_vehicle['Location Description'].isin(['APARTMENT'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
2345,20230675459_11,20230675459,D,DISPATCHED,O,OPEN,Open,11/10/2023 11:00:00 PM,11/11/2023 11:56:00 AM,BELL RD,...,1.0,1.0,30.0,15.0,2023-11-10,23:00:00,62,50,0.22,56.0
2372,20230674785_11,20230674785,S,SUSPECT,A,CLEARED BY ARREST,Closed,11/10/2023 10:11:00 PM,11/11/2023 12:40:00 AM,3417 3417,...,1.0,2.0,7.0,44.0,2023-11-10,22:11:00,62,50,0.22,56.0
3354,20230404118_11,20230404118,D,DISPATCHED,O,OPEN,Open,07/09/2023 10:00:00 PM,07/09/2023 10:00:00 PM,PARAGON MILLS RD,...,1.0,1.0,12.0,3.0,2023-07-09,22:00:00,80,70,0.12,75.0
5475,20230519106_12,20230519106,D,DISPATCHED,O,OPEN,Open,09/01/2023 02:13:00 AM,09/01/2023 09:25:00 AM,MASSMAN DR,...,1.0,1.0,6.0,22.0,2023-09-01,02:13:00,92,63,0.00,77.5
5476,20230519106_11,20230519106,D,DISPATCHED,O,OPEN,Open,09/01/2023 02:13:00 AM,09/01/2023 09:25:00 AM,MASSMAN DR,...,1.0,1.0,6.0,22.0,2023-09-01,02:13:00,92,63,0.00,77.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119686,20230223324_11,20230223324,D,DISPATCHED,O,OPEN,Open,04/16/2023 04:09:00 PM,04/16/2023 04:09:00 PM,SUGAR CREEK CIR,...,1.0,1.0,7.0,38.0,2023-04-16,16:09:00,72,49,0.03,60.5
119985,20230145882_11,20230145882,D,DISPATCHED,O,OPEN,Open,03/09/2023 04:00:00 AM,03/09/2023 05:27:00 PM,NaN,...,1.0,1.0,23.0,22.0,2023-03-09,04:00:00,61,45,0.00,53.0
121510,20230207008_11,20230207008,D,DISPATCHED,O,OPEN,Open,04/07/2023 08:00:00 PM,04/08/2023 02:05:00 PM,MURFREESBORO PKE,...,1.0,1.0,9.0,22.0,2023-04-07,20:00:00,60,47,0.02,53.5
121846,20230107592_11,20230107592,T,NaN,O,OPEN,Open,02/19/2023 08:00:00 PM,02/20/2023 09:31:00 AM,OLD MATTHEWS RD,...,1.0,1.0,2.0,16.0,2023-02-19,20:00:00,63,44,0.00,53.5


In [44]:
crime_vehicle.apt['Average Temp'].mean()

65.0

In [45]:
crime_vehicle.apt['Precip.'].mean()

0.12381818181818183

In [46]:
crime_vehicle.apt['Date'].value_counts()

Date
2023-05-13    5
2023-10-15    5
2023-09-30    4
2023-11-13    4
2023-07-04    3
             ..
2023-07-09    1
2023-09-03    1
2023-09-06    1
2023-08-26    1
2023-01-12    1
Name: count, Length: 78, dtype: int64

## ANALYSIS- SIMPLE ASSAULT

In [47]:
crime_assault =nashvillecrime_weather.loc[nashvillecrime_weather['Offense Description'].isin(['SIMPLE ASSLT'])]
crime_assault

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
25,20230636667_11,20230636667,S,SUSPECT,O,OPEN,Open,10/18/2023 10:00:00 PM,10/25/2023 10:28:00 AM,INTERSTATE DR,...,1.0,1.0,14.0,19.0,2023-10-18,22:00:00,73,41,0.00,57.0
123,20230621542_11,20230621542,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,10/18/2023 09:00:00 PM,10/18/2023 10:36:00 PM,339 339,...,1.0,2.0,27.0,26.0,2023-10-18,21:00:00,73,41,0.00,57.0
124,20230621539_11,20230621539,D,DISPATCHED,O,OPEN,Open,10/18/2023 09:30:00 PM,10/18/2023 10:55:00 PM,HERMAN ST,...,1.0,1.0,20.0,40.0,2023-10-18,21:30:00,73,41,0.00,57.0
153,20230621250_11,20230621250,D,DISPATCHED,A,CLEARED BY ARREST,Closed,10/18/2023 05:00:00 PM,10/18/2023 10:52:00 PM,727 727,...,1.0,1.0,30.0,15.0,2023-10-18,17:00:00,73,41,0.00,57.0
183,20230620958_11,20230620958,D,DISPATCHED,O,OPEN,Open,10/18/2023 05:11:00 PM,10/18/2023 05:38:00 PM,DR MARTIN L KING JR,...,1.0,1.0,20.0,29.0,2023-10-18,17:11:00,73,41,0.00,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122250,20230025300_11,20230025300,D,DISPATCHED,A,CLEARED BY ARREST,Closed,01/12/2023 07:00:00 PM,01/13/2023 12:15:00 AM,5548 5548,...,1.0,1.0,30.0,15.0,2023-01-12,19:00:00,69,40,0.31,54.5
122255,20230025228_11,20230025228,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,01/12/2023 07:14:00 PM,01/12/2023 08:02:00 PM,2929 2929,...,1.0,1.0,30.0,15.0,2023-01-12,19:14:00,69,40,0.31,54.5
122281,20230024909_11,20230024909,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,01/12/2023 04:50:00 PM,01/12/2023 07:04:00 PM,1330 1330,...,1.0,1.0,6.0,22.0,2023-01-12,16:50:00,69,40,0.31,54.5
122392,20230023498_31,20230023498,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:30:00 AM,01/12/2023 03:08:00 AM,AMALIE DR,...,1.0,1.0,21.0,3.0,2023-01-12,01:30:00,69,40,0.31,54.5


In [48]:
crime_assault['Average Temp'].value_counts()

Average Temp
82.5    155
78.0    146
73.5    145
72.5    139
79.5    138
       ... 
58.0     16
86.5     14
84.5     13
42.5     12
31.0     12
Name: count, Length: 101, dtype: int64

In [49]:
crime_assault['Date'].value_counts()

Date
2023-08-13    33
2023-06-24    33
2023-08-19    31
2023-01-22    31
2023-11-25    31
              ..
2023-01-10     8
2023-01-11     7
2023-02-16     7
2023-02-02     7
2023-08-24     6
Name: count, Length: 348, dtype: int64

In [50]:
crime_assault['Location Description'].value_counts()

Location Description
RESIDENCE, HOME                       2229
APARTMENT                             1385
PARKING LOT, GARAGE                    651
HIGHWAY, ROAD, ALLEY                   598
BAR, NIGHT CLUB                        227
HOTEL, MOTEL, ETC.                     200
RESTAURANT                             157
CONVENIENCE STORE                      111
DEPARTMENT, DISCOUNT STORE             100
SERVICE, GAS STATION                    72
AIR, BUS, TRAIN TERMINAL                62
COMMERCIAL, OFFICE BUILDING             56
School-Elementary/Secondary             53
SPECIALTY STORE                         53
HOSPITAL                                45
GROCERY, SUPERMARKET                    42
PARK                                    41
Shelter-Mission/Homeless                28
Arena/Stadium/Fairgrounds/Coliseum      27
Shopping Mall                           20
GOVERNMENT, PUBLIC BUILDING             19
OTHER, UNKNOWN                          12
FIELD, WOODS                     

In [51]:
crime_assault.res=crime_assault.loc[crime_assault['Location Description'].isin(['RESIDENCE, HOME'])]
crime_assault.res

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\3397514441.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_assault.res=crime_assault.loc[crime_assault['Location Description'].isin(['RESIDENCE, HOME'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
123,20230621542_11,20230621542,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,10/18/2023 09:00:00 PM,10/18/2023 10:36:00 PM,339 339,...,1.0,2.0,27.0,26.0,2023-10-18,21:00:00,73,41,0.00,57.0
153,20230621250_11,20230621250,D,DISPATCHED,A,CLEARED BY ARREST,Closed,10/18/2023 05:00:00 PM,10/18/2023 10:52:00 PM,727 727,...,1.0,1.0,30.0,15.0,2023-10-18,17:00:00,73,41,0.00,57.0
208,20230620727_22,20230620727,D,DISPATCHED,O,OPEN,Open,10/18/2023 03:00:00 PM,10/18/2023 03:57:00 PM,SPENCER AVE,...,1.0,1.0,24.0,46.0,2023-10-18,15:00:00,73,41,0.00,57.0
229,20230620535_11,20230620535,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,10/18/2023 01:40:00 PM,10/18/2023 02:09:00 PM,1007 1007,...,1.0,1.0,3.0,23.0,2023-10-18,13:40:00,73,41,0.00,57.0
273,20230620071_11,20230620071,D,DISPATCHED,O,OPEN,Open,10/18/2023 10:30:00 AM,10/18/2023 11:15:00 AM,CHADWELL DR,...,1.0,1.0,3.0,12.0,2023-10-18,10:30:00,73,41,0.00,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121901,20230106625_11,20230106625,D,DISPATCHED,A,CLEARED BY ARREST,Closed,02/19/2023 08:24:00 PM,02/19/2023 09:22:00 PM,620 620,...,1.0,1.0,14.0,4.0,2023-02-19,20:24:00,63,44,0.00,53.5
122086,20230105628_11,20230105628,D,DISPATCHED,O,OPEN,Open,02/19/2023 02:30:00 AM,02/19/2023 03:46:00 AM,MORENA ST,...,1.0,1.0,25.0,51.0,2023-02-19,02:30:00,63,44,0.00,53.5
122232,20230025427_11,20230025427,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,01/12/2023 08:40:00 PM,01/12/2023 09:59:00 PM,253 253,...,1.0,1.0,8.0,38.0,2023-01-12,20:40:00,69,40,0.31,54.5
122250,20230025300_11,20230025300,D,DISPATCHED,A,CLEARED BY ARREST,Closed,01/12/2023 07:00:00 PM,01/13/2023 12:15:00 AM,5548 5548,...,1.0,1.0,30.0,15.0,2023-01-12,19:00:00,69,40,0.31,54.5


In [52]:
crime_assault.res['Average Temp'].mean()

64.19807088380439

In [53]:
crime_assault.res['Precip.'].mean()

0.09314939434724091

In [54]:
crime_assault.res['Date'].value_counts()

Date
2023-01-22    19
2023-06-24    18
2023-04-08    17
2023-11-19    16
2023-11-25    15
              ..
2023-10-12     1
2023-02-16     1
2023-06-05     1
2023-07-06     1
2023-01-11     1
Name: count, Length: 348, dtype: int64

In [56]:
crime_assault.apt=crime_assault.loc[crime_assault['Location Description'].isin(['APARTMENT'])]
crime_assault.apt

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\3920879187.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_assault.apt=crime_assault.loc[crime_assault['Location Description'].isin(['APARTMENT'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
124,20230621539_11,20230621539,D,DISPATCHED,O,OPEN,Open,10/18/2023 09:30:00 PM,10/18/2023 10:55:00 PM,HERMAN ST,...,1.0,1.0,20.0,40.0,2023-10-18,21:30:00,73,41,0.00,57.0
241,20230620461_11,20230620461,D,DISPATCHED,O,OPEN,Open,10/18/2023 01:00:00 PM,10/18/2023 02:36:00 PM,UNIVERSITY,...,1.0,1.0,18.0,11.0,2023-10-18,13:00:00,73,41,0.00,57.0
306,20230619650_12,20230619650,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,10/18/2023 06:50:00 AM,10/18/2023 08:23:00 AM,2400 2400,...,1.0,1.0,2.0,34.0,2023-10-18,06:50:00,73,41,0.00,57.0
307,20230619650_11,20230619650,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,10/18/2023 06:50:00 AM,10/18/2023 08:23:00 AM,2400 2400,...,1.0,1.0,2.0,34.0,2023-10-18,06:50:00,73,41,0.00,57.0
542,20230322002_11,20230322002,D,DISPATCHED,O,OPEN,Open,06/01/2023 11:00:00 PM,06/02/2023 12:44:00 AM,APACHE TRL,...,1.0,1.0,28.0,3.0,2023-06-01,23:00:00,88,67,0.00,77.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121910,20230106591_11,20230106591,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,02/19/2023 07:40:00 PM,02/19/2023 10:18:00 PM,1501 1501,...,1.0,1.0,20.0,51.0,2023-02-19,19:40:00,63,44,0.00,53.5
121928,20230106495_11,20230106495,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,02/19/2023 05:30:00 PM,02/19/2023 07:01:00 PM,315 315,...,1.0,1.0,13.0,4.0,2023-02-19,17:30:00,63,44,0.00,53.5
122184,20230026707_11,20230026707,D,DISPATCHED,A,CLEARED BY ARREST,Closed,01/12/2023 09:00:00 PM,01/13/2023 12:39:00 PM,900 900,...,1.0,1.0,29.0,3.0,2023-01-12,21:00:00,69,40,0.31,54.5
122255,20230025228_11,20230025228,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,01/12/2023 07:14:00 PM,01/12/2023 08:02:00 PM,2929 2929,...,1.0,1.0,30.0,15.0,2023-01-12,19:14:00,69,40,0.31,54.5


In [57]:
crime_assault.apt['Date'].value_counts()

Date
2023-03-05    13
2023-05-17    12
2023-02-28    10
2023-09-11    10
2023-08-31     9
              ..
2023-03-20     1
2023-03-30     1
2023-08-24     1
2023-09-14     1
2023-08-11     1
Name: count, Length: 337, dtype: int64

In [58]:
crime_assault.apt['Average Temp'].value_counts()

Average Temp
73.5    36
82.5    35
77.0    34
75.0    34
72.5    32
        ..
43.5     2
31.0     1
45.5     1
42.5     1
32.5     1
Name: count, Length: 101, dtype: int64

In [59]:
crime_assault.apt['Average Temp'].mean()

64.80216606498195

In [60]:
crime_assault.apt['Precip.'].mean()

0.10957400722021662

In [61]:
crime_assault.apt['Date'].value_counts()

Date
2023-03-05    13
2023-05-17    12
2023-02-28    10
2023-09-11    10
2023-08-31     9
              ..
2023-03-20     1
2023-03-30     1
2023-08-24     1
2023-09-14     1
2023-08-11     1
Name: count, Length: 337, dtype: int64

In [62]:
crime_assault.lot=crime_assault.loc[crime_assault['Location Description'].isin(['PARKING LOT, GARAGE'])]
crime_assault.lot

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\2587050734.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_assault.lot=crime_assault.loc[crime_assault['Location Description'].isin(['PARKING LOT, GARAGE'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
259,20230620264_12,20230620264,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,10/18/2023 10:40:00 AM,10/18/2023 12:00:00 PM,411 411,...,1.0,1.0,18.0,11.0,2023-10-18,10:40:00,73,41,0.00,57.0
260,20230620264_11,20230620264,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,10/18/2023 10:40:00 AM,10/18/2023 12:00:00 PM,411 411,...,1.0,1.0,18.0,11.0,2023-10-18,10:40:00,73,41,0.00,57.0
896,20230203302_11,20230203302,D,DISPATCHED,O,OPEN,Open,04/06/2023 06:08:00 PM,04/06/2023 09:07:00 PM,GALLATIN,...,1.0,1.0,4.0,12.0,2023-04-06,18:08:00,63,46,1.40,54.5
1379,20230652539_11,20230652539,D,DISPATCHED,O,OPEN,Open,11/01/2023 11:30:00 AM,11/01/2023 12:39:00 PM,ROSA L PARKS BLVD,...,1.0,1.0,20.0,51.0,2023-11-01,11:30:00,51,30,0.00,40.5
2225,20230718339_11,20230718339,D,DISPATCHED,A,CLEARED BY ARREST,Closed,12/01/2023 01:30:00 AM,12/01/2023 04:16:00 AM,1584 1584,...,1.0,2.0,17.0,12.0,2023-12-01,01:30:00,61,45,0.58,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121550,20230206378_11,20230206378,D,DISPATCHED,A,CLEARED BY ARREST,Closed,04/07/2023 10:00:00 PM,04/08/2023 04:40:00 AM,101 101,...,1.0,1.0,13.0,16.0,2023-04-07,22:00:00,60,47,0.02,53.5
121566,20230205962_11,20230205962,D,DISPATCHED,A,CLEARED BY ARREST,Closed,04/07/2023 10:48:00 PM,04/08/2023 12:31:00 AM,333 333,...,1.0,1.0,4.0,12.0,2023-04-07,22:48:00,60,47,0.02,53.5
121575,20230205851_11,20230205851,D,DISPATCHED,O,OPEN,Open,04/07/2023 12:00:00 PM,04/07/2023 11:22:00 PM,LAFAYETTE ST,...,1.0,1.0,20.0,40.0,2023-04-07,12:00:00,60,47,0.02,53.5
121889,20230106724_11,20230106724,D,DISPATCHED,A,CLEARED BY ARREST,Closed,02/19/2023 09:30:00 PM,02/20/2023 12:25:00 AM,7158 7158,...,1.0,1.0,32.0,46.0,2023-02-19,21:30:00,63,44,0.00,53.5


In [63]:
crime_assault.lot['Average Temp'].value_counts()

Average Temp
82.5    19
72.5    19
67.5    18
78.0    18
52.5    16
        ..
53.0     1
55.5     1
49.5     1
39.5     1
67.0     1
Name: count, Length: 96, dtype: int64

In [64]:
crime_assault.lot['Average Temp'].mean()

65.06605222734255

In [65]:
crime_assault.lot['Precip.'].mean()

0.08998463901689707

In [66]:
crime_assault.lot['Date'].value_counts()

Date
2023-07-30    9
2023-08-20    8
2023-06-02    7
2023-08-27    7
2023-11-18    6
             ..
2023-05-24    1
2023-11-08    1
2023-09-12    1
2023-01-07    1
2023-01-23    1
Name: count, Length: 276, dtype: int64

In [67]:
crime_assault.hra=crime_assault.loc[crime_assault['Location Description'].isin(['HIGHWAY, ROAD, ALLEY'])]
crime_assault.hra

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\3687392417.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_assault.hra=crime_assault.loc[crime_assault['Location Description'].isin(['HIGHWAY, ROAD, ALLEY'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
401,20230416771_11,20230416771,D,DISPATCHED,O,OPEN,Open,06/01/2023 02:00:00 AM,07/15/2023 11:59:00 PM,NOLENSVILLE PKE,...,1.0,1.0,21.0,3.0,2023-06-01,02:00:00,88,67,0.00,77.5
1317,20230653049_12,20230653049,D,DISPATCHED,O,OPEN,Open,11/01/2023 05:36:00 PM,11/01/2023 05:36:00 PM,MURFREESBORO PKE,...,1.0,1.0,23.0,22.0,2023-11-01,17:36:00,51,30,0.00,40.5
1318,20230653049_11,20230653049,D,DISPATCHED,O,OPEN,Open,11/01/2023 05:36:00 PM,11/01/2023 05:36:00 PM,MURFREESBORO PKE,...,1.0,1.0,23.0,22.0,2023-11-01,17:36:00,51,30,0.00,40.5
1762,20230378457_11,20230378457,D,DISPATCHED,O,OPEN,Open,06/28/2023 07:57:00 AM,06/28/2023 08:48:00 AM,ELLISTON ST,...,1.0,2.0,27.0,26.0,2023-06-28,07:57:00,93,67,0.00,80.0
2043,20230719954_11,20230719954,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,12/01/2023 09:00:00 PM,12/01/2023 11:44:00 PM,710 710,...,1.0,1.0,14.0,4.0,2023-12-01,21:00:00,61,45,0.58,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121075,20230193601_11,20230193601,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,04/02/2023 02:45:00 AM,04/02/2023 08:13:00 PM,3RD & SYMPHONY,...,1.0,1.0,20.0,29.0,2023-04-02,02:45:00,69,39,0.00,54.0
121648,20230205286_11,20230205286,D,DISPATCHED,A,CLEARED BY ARREST,Closed,04/07/2023 04:00:00 PM,04/07/2023 06:03:00 PM,100 100,...,1.0,1.0,14.0,19.0,2023-04-07,16:00:00,60,47,0.02,53.5
121753,20230204503_21,20230204503,D,DISPATCHED,O,OPEN,Open,04/07/2023 03:00:00 AM,04/07/2023 07:40:00 AM,3RD,...,1.0,1.0,20.0,40.0,2023-04-07,03:00:00,60,47,0.02,53.5
122174,20230026871_11,20230026871,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:00:00 PM,01/13/2023 12:51:00 PM,DANBY DR,...,1.0,1.0,12.0,3.0,2023-01-12,13:00:00,69,40,0.31,54.5


In [68]:
crime_assault.hra['Average Temp'].value_counts()

Average Temp
69.5    21
61.0    19
54.5    18
76.0    18
77.0    18
        ..
64.5     1
67.0     1
61.5     1
80.0     1
85.0     1
Name: count, Length: 93, dtype: int64

In [69]:
crime_assault.hra['Precip.'].mean()

0.11963210702341136

In [70]:
crime_assault.hra['Date'].value_counts()

Date
2023-08-10    8
2023-09-26    7
2023-01-18    6
2023-04-15    6
2023-10-15    6
             ..
2023-01-31    1
2023-07-14    1
2023-09-11    1
2023-09-20    1
2023-06-01    1
Name: count, Length: 271, dtype: int64

In [71]:
crime_assault.bnc=crime_assault.loc[crime_assault['Location Description'].isin(['BAR, NIGHT CLUB'])]
crime_assault.bnc

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\3218114738.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_assault.bnc=crime_assault.loc[crime_assault['Location Description'].isin(['BAR, NIGHT CLUB'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
883,20230203417_11,20230203417,D,DISPATCHED,A,CLEARED BY ARREST,Closed,04/06/2023 07:30:00 PM,04/06/2023 11:10:00 PM,1911 1911,...,1.0,1.0,20.0,40.0,2023-04-06,19:30:00,63,46,1.40,54.5
1199,20230653999_11,20230653999,D,DISPATCHED,O,OPEN,Open,11/01/2023 11:00:00 PM,11/02/2023 12:42:00 AM,6TH AV N,...,1.0,1.0,20.0,51.0,2023-11-01,23:00:00,51,30,0.00,40.5
1791,20230378033_16,20230378033,D,DISPATCHED,O,OPEN,Open,06/28/2023 02:30:00 AM,06/28/2023 03:36:00 AM,BROADWAY,...,1.0,1.0,20.0,40.0,2023-06-28,02:30:00,93,67,0.00,80.0
1792,20230378033_15,20230378033,D,DISPATCHED,O,OPEN,Open,06/28/2023 02:30:00 AM,06/28/2023 03:36:00 AM,BROADWAY,...,1.0,1.0,20.0,40.0,2023-06-28,02:30:00,93,67,0.00,80.0
1793,20230378033_14,20230378033,D,DISPATCHED,O,OPEN,Open,06/28/2023 02:30:00 AM,06/28/2023 03:36:00 AM,BROADWAY,...,1.0,1.0,20.0,40.0,2023-06-28,02:30:00,93,67,0.00,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120301,20230115384_11,20230115384,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,02/23/2023 04:00:00 PM,02/23/2023 05:00:00 PM,408 408,...,1.0,1.0,20.0,29.0,2023-02-23,16:00:00,85,55,0.00,70.0
120631,20230103073_11,20230103073,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,02/17/2023 07:20:00 PM,02/17/2023 07:20:00 PM,316 316,...,1.0,1.0,20.0,29.0,2023-02-17,19:20:00,44,27,0.01,35.5
121263,20230072599_11,20230072599,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,02/03/2023 08:28:00 PM,02/03/2023 08:58:00 PM,422 422,...,1.0,1.0,20.0,40.0,2023-02-03,20:28:00,36,24,0.00,30.0
121440,20230071011_12,20230071011,D,DISPATCHED,O,OPEN,Open,02/03/2023 01:30:00 AM,02/03/2023 02:26:00 AM,APACHE TRL,...,1.0,1.0,28.0,3.0,2023-02-03,01:30:00,36,24,0.00,30.0


In [72]:
crime_assault.bnc['Average Temp'].value_counts()

Average Temp
64.0    11
80.5    11
54.5     8
40.0     7
44.5     7
        ..
75.5     1
36.5     1
48.5     1
62.5     1
49.5     1
Name: count, Length: 68, dtype: int64

In [73]:
crime_assault.bnc['Precip.'].mean()

0.1317180616740088

In [74]:
crime_assault.bnc['Date'].value_counts()

Date
2023-07-08    9
2023-01-07    7
2023-06-28    6
2023-10-11    4
2023-04-21    4
             ..
2023-04-03    1
2023-06-19    1
2023-11-17    1
2023-01-22    1
2023-01-05    1
Name: count, Length: 125, dtype: int64

In [75]:
crime_assault.hme=crime_assault.loc[crime_assault['Location Description'].isin(['HOTEL, MOTEL, ETC.'])]
crime_assault.hme

C:\Users\antoi\AppData\Local\Temp\ipykernel_1184\1252318262.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  crime_assault.hme=crime_assault.loc[crime_assault['Location Description'].isin(['HOTEL, MOTEL, ETC.'])]


,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
2049,20230719936_11,20230719936,D,DISPATCHED,O,OPEN,Open,12/01/2023 09:38:00 PM,12/01/2023 09:38:00 PM,4TH AV S,...,1.0,1.0,20.0,40.0,2023-12-01,21:38:00,61,45,0.58,53.0
2580,20230673302_12,20230673302,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,11/10/2023 01:00:00 AM,11/10/2023 05:21:00 AM,305 305,...,1.0,1.0,20.0,29.0,2023-11-10,01:00:00,62,50,0.22,56.0
2581,20230673302_11,20230673302,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,11/10/2023 01:00:00 AM,11/10/2023 05:21:00 AM,305 305,...,1.0,1.0,20.0,29.0,2023-11-10,01:00:00,62,50,0.22,56.0
4767,20230644958_11,20230644958,D,DISPATCHED,A,CLEARED BY ARREST,Closed,10/28/2023 10:38:00 PM,10/29/2023 12:44:00 AM,222 222,...,1.0,1.0,20.0,29.0,2023-10-28,22:38:00,87,66,0.00,76.5
5519,20230518489_11,20230518489,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,09/01/2023 12:30:00 AM,09/01/2023 01:17:00 AM,1801 1801,...,1.0,1.0,13.0,16.0,2023-09-01,00:30:00,92,63,0.00,77.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121870,20230106858_11,20230106858,D,DISPATCHED,O,OPEN,Open,02/19/2023 10:45:00 PM,02/19/2023 11:42:00 PM,REP JOHN LEWIS WAY S,...,1.0,1.0,20.0,40.0,2023-02-19,22:45:00,63,44,0.00,53.5
121932,20230106484_12,20230106484,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,02/19/2023 05:56:00 PM,02/19/2023 07:14:00 PM,2511 2511,...,1.0,1.0,8.0,38.0,2023-02-19,17:56:00,63,44,0.00,53.5
121933,20230106484_11,20230106484,D,DISPATCHED,R,REFUSED TO COOPERATE,Closed,02/19/2023 05:56:00 PM,02/19/2023 07:14:00 PM,2511 2511,...,1.0,1.0,8.0,38.0,2023-02-19,17:56:00,63,44,0.00,53.5
122034,20230105953_11,20230105953,D,DISPATCHED,O,OPEN,Open,02/19/2023 09:00:00 AM,02/19/2023 10:01:00 AM,ROYAL PKWY,...,1.0,1.0,6.0,38.0,2023-02-19,09:00:00,63,44,0.00,53.5


In [76]:
crime_assault.hme['Average Temp'].value_counts()

Average Temp
76.0    8
53.5    7
64.5    7
83.0    6
35.5    6
       ..
85.0    1
49.0    1
61.0    1
87.0    1
54.0    1
Name: count, Length: 77, dtype: int64

In [77]:
crime_assault.hme['Precip.'].mean()

0.106

In [78]:
crime_assault.hme['Date'].value_counts()

Date
2023-02-15    6
2023-02-04    5
2023-02-19    5
2023-09-15    4
2023-08-14    4
             ..
2023-09-14    1
2023-05-17    1
2023-01-07    1
2023-11-20    1
2023-08-26    1
Name: count, Length: 136, dtype: int64

## ANALYSIS- VEHICLE THEFT

In [79]:
crime_vehtheft =nashvillecrime_weather.loc[nashvillecrime_weather['Offense Description'].isin(['VEHICLE THEFT'])]
crime_vehtheft

,Primary Key,Incident Number,Report Type,Report Type Description,Incident Status Code,Incident Status Description,Investigation Status,Incident Occurred,Incident Reported,Incident Location,...,Davidson County Border (GIS),Service Districts (GIS),Council District Outlines (GIS),Zip Codes (GIS),Date,Time,High,Low,Precip.,Average Temp
35,20230629666_11,20230629666,D,DISPATCHED,O,OPEN,Open,10/18/2023 03:00:00 PM,10/22/2023 06:36:00 PM,BLUE LAKE CIR,...,1.0,1.0,29.0,15.0,2023-10-18,15:00:00,73,41,0.00,57.0
45,20230625793_11,20230625793,T,NaN,O,OPEN,Open,10/18/2023 05:00:00 PM,10/20/2023 01:48:00 PM,HICKORY TRACE DR,...,1.0,1.0,21.0,3.0,2023-10-18,17:00:00,73,41,0.00,57.0
60,20230623687_11,20230623687,D,DISPATCHED,O,OPEN,Open,10/18/2023 03:30:00 AM,10/19/2023 08:19:00 PM,VILLAGE HILLS DR,...,1.0,1.0,23.0,22.0,2023-10-18,03:30:00,73,41,0.00,57.0
62,20230623514_11,20230623514,D,DISPATCHED,O,OPEN,Open,10/18/2023 03:00:00 AM,10/19/2023 05:55:00 PM,ALBION ST,...,1.0,1.0,25.0,51.0,2023-10-18,03:00:00,73,41,0.00,57.0
88,20230622571_11,20230622571,D,DISPATCHED,O,OPEN,Open,10/18/2023 06:30:00 PM,10/19/2023 09:18:00 AM,HARDING PKE,...,1.0,1.0,3.0,12.0,2023-10-18,18:30:00,73,41,0.00,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122153,20230027820_11,20230027820,D,DISPATCHED,O,OPEN,Open,01/12/2023 06:30:00 AM,01/14/2023 01:31:00 AM,BELL RD,...,1.0,1.0,29.0,3.0,2023-01-12,06:30:00,69,40,0.31,54.5
122177,20230026851_11,20230026851,D,DISPATCHED,O,OPEN,Open,01/12/2023 09:00:00 PM,01/13/2023 12:58:00 PM,COPPERFIELD WAY,...,1.0,1.0,5.0,31.0,2023-01-12,21:00:00,69,40,0.31,54.5
122327,20230024539_11,20230024539,D,DISPATCHED,O,OPEN,Open,01/12/2023 01:52:00 PM,01/12/2023 01:52:00 PM,HAMBLEN,...,1.0,2.0,17.0,12.0,2023-01-12,13:52:00,69,40,0.31,54.5
122338,20230024473_11,20230024473,D,DISPATCHED,O,OPEN,Open,01/12/2023 12:00:00 PM,01/12/2023 01:11:00 PM,NOLENSVILLE PKE,...,1.0,1.0,21.0,3.0,2023-01-12,12:00:00,69,40,0.31,54.5


In [80]:
crime_vehtheft['Location Description'].value_counts()

Location Description
PARKING LOT, GARAGE                  3063
RESIDENCE, HOME                       867
HIGHWAY, ROAD, ALLEY                  656
APARTMENT                             219
Auto Dealership New/Used               88
SERVICE, GAS STATION                   49
HOTEL, MOTEL, ETC.                     32
CONVENIENCE STORE                      18
COMMERCIAL, OFFICE BUILDING            15
RESTAURANT                             11
CONSTRUCTION SITE                      11
SPECIALTY STORE                        10
PARK                                    9
BAR, NIGHT CLUB                         7
OTHER, UNKNOWN                          6
RENTAL STORAGE FACILITY                 6
Auto Salvage/Junkyard                   5
CHURCH, SYNAGOGUE, TEMPLE               5
GROCERY, SUPERMARKET                    5
HOSPITAL                                4
DEPARTMENT, DISCOUNT STORE              3
Shelter-Mission/Homeless                2
Carport                                 2
Farm Facility